В этом ноутбуке разрабатывается система поиска похожих изображений растений на основе входного изображения.

## Содержание
1. Настройка окружения
2. Загрузка и подготовка данных
3. Модель для извлечения признаков
4. Создание векторов признаков для библиотеки изображений
5. Реализация функции поиска похожих изображений
6. Оценка модели
7. Визуализация результатов (для 5 случайных изображений)
8. Сохранение модели для использования в API

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm.notebook import tqdm
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [3]:
#Фиксирую seed для воспроизводимости
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
#Проверка доступности GPU, в обратном случае использую CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda
